In [14]:
from loading_data import queryDB
import pandas as pd

In [15]:
import json
import numpy as np
import copy
import os

config_path = 'config.json'

with open(config_path) as config_buffer:
   config = json.loads(config_buffer.read())

model_path = config['model_weights']
num_concepts = len(config['conceptids'])
class_map_file = config['class_map']
concepts = config['conceptids']
classmap = pd.read_csv(class_map_file, header=None).to_dict()[0]

In [16]:
queryDB("select * from videos where filename='DocRicketts-0701_20141216T212020Z_00-48-12-26TC_h264.mp4'")

,id,filename,gpsstart,gpsstop,starttime,endtime,startdepth,enddepth,description
0,8,DocRicketts-0701_20141216T212020Z_00-48-12-26T...,"(36.798367,-122.108671)","(36.798215,-122.108195)",2014-12-16 21:20:20,2014-12-16 21:36:20,217,149,


In [60]:
annotations = queryDB('select * from annotations where videoid=8 and userid!=17 and timeinvideo > 160 and timeinvideo < 190')

In [61]:
annotations.head()

,id,videoid,userid,conceptid,timeinvideo,x1,y1,x2,y2,videowidth,videoheight,dateannotated,image,imagewithbox,comment,unsure,originalid,framenum
0,433,8,6,236,178.732221,575.006251,49.006271,698.999957,155.993774,1280.0,720.0,2018-09-05,433.png,433_box.png,None,False,433,None
1,427,8,6,236,176.381214,740.006190,56.006271,842.999934,156.993767,1280.0,720.0,2018-09-05,427.png,427_box.png,None,False,427,None
2,434,8,6,236,178.485498,1116.006251,50.006271,1239.999995,156.993767,1280.0,720.0,2018-09-05,434.png,434_box.png,None,False,434,None
3,420,8,6,236,172.134367,1049.006251,203.006271,1179.993739,317.006248,1280.0,720.0,2018-09-05,420.png,420_box.png,None,False,420,None
4,422,8,6,236,172.134367,833.006251,2.006256,963.993739,116.006248,1280.0,720.0,2018-09-05,422.png,422_box.png,None,False,422,None


In [19]:
def compute_overlap(annotationA, annotationB):
    # if there is no overlap in x dimension
    if annotationB.x2 - annotationA.x1 < 0 or annotationA.x2 - annotationB.x1 < 0:
        return 0
    # if there is no overlap in y dimension
    if annotationB.y2 - annotationA.y1 < 0 or annotationA.y2 - annotationB.y1 < 0:
        return 0
    
    areaA = (annotationA.x2-annotationA.x1) * (annotationA.y2-annotationA.y1)
    areaB = (annotationB.x2-annotationB.x1) * (annotationB.y2-annotationB.y1)

    width = min(annotationA.x2,annotationB.x2) - min(annotationA.x1,annotationB.x1)
    height = min(annotationA.y2,annotationB.y2) - min(annotationA.y1,annotationB.y1)
    
    area_intersect = height * width
    iou = area_intersect / (areaA + areaB - area_intersect)
    
    return iou

In [20]:
a = annotations.iloc[0]
a.x1 = -1
a.x2 = 0
if compute_overlap(a,annotations.iloc[0]) != 0:
    print("ERROR")
    
a = annotations.iloc[0]
a.x1 = 0
a.x2 = 1000
a.y1 = 0
a.y2 = 1000
b = annotations.iloc[0]
b.x1 = 0
b.x2 = 1000
b.y1 = 0
b.y2 = 500

if compute_overlap(a,b) != .5:
    print("ERROR")

/Users/kylemaxwell/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [21]:
fps = 29.97002997002997

In [22]:
#DOESNT WORK IF VAL IS MISSING

In [23]:
#is it false positive if something is misidentified, or false positive as well as false negative?
# currently both!

In [24]:
annotations.tail()

,id,videoid,userid,conceptid,timeinvideo,x1,y1,x2,y2,videowidth,videoheight,dateannotated,image,imagewithbox,comment,unsure,originalid,framenum
6,409,8,6,236,166.554130,93.006251,95.006241,225.006251,209.999985,1280.0,720.0,2018-09-05,409.png,409_box.png,None,False,409,None
7,407,8,6,236,166.554130,163.006251,195.006241,266.993747,294.006233,1280.0,720.0,2018-09-05,407.png,407_box.png,None,False,407,None
8,415,8,6,236,169.647763,961.006190,47.006256,1081.006183,150.993752,1280.0,720.0,2018-09-05,415.png,415_box.png,None,False,415,None
9,412,8,6,236,169.647763,63.006251,246.006241,204.006251,370.993736,1280.0,720.0,2018-09-05,412.png,412_box.png,None,False,412,None
10,408,8,6,236,166.554130,247.006251,140.006241,365.993762,239.999985,1280.0,720.0,2018-09-05,408.png,408_box.png,None,False,408,None


In [25]:
results = pd.read_csv('results.csv')

In [26]:
len(results.objectid.value_counts())

52

In [27]:
len(annotations)

11

In [28]:
results.frame_num = results.frame_num+160 * 30

AttributeError: 'DataFrame' object has no attribute 'frame_num'

In [29]:
results.head()

,Unnamed: 0,x1,y1,x2,y2,label,confidence,objectid,timeinvideo
0,0,477.47937,0.0,640.0,128.515671,3.0,0.546251,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,0.00
1,1,477.00000,0.0,640.0,129.000000,NaN,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,0.03
2,2,476.00000,3.0,639.0,132.000000,NaN,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,0.07
3,3,475.00000,8.0,638.0,137.000000,NaN,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,0.10
4,4,472.00000,11.0,635.0,140.000000,NaN,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,0.13


In [41]:
predicted_frames = results[results.confidence.notnull()]

In [43]:
#predicted_frames

In [96]:
# Add the proper label to all frames
label = None
objects = results.groupby(['objectid'])
for oid, result in objects:
    scores = {}
    for k , label in result.groupby(['label']):
        scores[k] = label.confidence.mean()
    idmax = max(scores.keys(), key=(lambda k: scores[k]))
    results.loc[results.objectid == oid,'label'] = idmax
results['label'] = results['label'].apply(lambda x: concepts[int(x)])
results['conceptid'] = results['label']

IndexError: list index out of range

In [97]:
results.head()

,id,x1,y1,x2,y2,label,confidence,objectid,timeinvideo,conceptid,videoid
0,0,477.47937,0.0,640.0,128.515671,383,0.546251,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,0.00,383,8
1,1,477.00000,0.0,640.0,129.000000,383,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,0.03,383,8
2,2,476.00000,3.0,639.0,132.000000,383,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,0.07,383,8
3,3,475.00000,8.0,638.0,137.000000,383,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,0.10,383,8
4,4,472.00000,11.0,635.0,140.000000,383,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,0.13,383,8


In [95]:
grouped = results.groupby(['objectid']).label.mean().reset_index()
counts = grouped.groupby('label').count()
counts.columns = ['pred_num']
counts

,pred_num
label,
236,49
383,3


In [92]:
groundtruth_counts = pd.DataFrame(annotations.groupby('conceptid').id.count())
groundtruth_counts.columns = ['true_num']
groundtruth_counts

,true_num
conceptid,
236,54
237,2
383,2


In [94]:
df = pd.concat((counts, groundtruth_counts), axis=1, join='outer').fillna(0)
df['accuracy'] = (df.true_num - (df.true_num - df.pred_num).abs()) / df.true_num
df

,pred_num,true_num,accuracy
236,49.0,54,0.907407
237,0.0,2,0.000000
383,3.0,2,0.500000


In [32]:
temp = list(results.columns)
temp[0] = 'id'
results.columns = temp

In [34]:
results.head()

,id,x1,y1,x2,y2,label,confidence,objectid,timeinvideo,conceptid
0,0,477.47937,0.0,640.0,128.515671,383,0.546251,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,0.00,383
1,1,477.00000,0.0,640.0,129.000000,383,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,0.03,383
2,2,476.00000,3.0,639.0,132.000000,383,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,0.07,383
3,3,475.00000,8.0,638.0,137.000000,383,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,0.10,383
4,4,472.00000,11.0,635.0,140.000000,383,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,0.13,383


In [38]:
annotations['frame_num'] = np.rint(annotations['timeinvideo'] * fps)
annotations.head()

,id,videoid,userid,conceptid,timeinvideo,x1,y1,x2,y2,videowidth,videoheight,dateannotated,image,imagewithbox,comment,unsure,originalid,framenum,frame_num
0,411,8,6,236,169.627744,1198.006251,272.006241,1279.000000,377.999985,1280.0,720.0,2018-09-05,411.png,411_box.png,None,False,411,None,5084.0
1,414,8,6,236,169.647763,215.006236,96.006256,330.993731,199.993752,1280.0,720.0,2018-09-05,414.png,414_box.png,None,False,414,None,5084.0
2,413,8,6,236,169.647763,103.006251,157.006241,244.006236,281.993752,1280.0,720.0,2018-09-05,413.png,413_box.png,None,False,413,None,5084.0
3,410,8,6,237,169.627744,1057.006251,210.006241,1279.000000,358.993729,1280.0,720.0,2018-09-05,410.png,410_box.png,None,False,410,None,5084.0
4,404,8,6,383,160.694075,890.006251,100.006256,1166.006221,305.000015,1280.0,720.0,2018-09-05,404.png,404_box.png,None,False,404,None,4816.0


In [36]:
def score_predictions(validation,predictions,iou_thresh, concepts, fps):
    # group predictions by video frames
    predictions = predictions.groupby(['videoid','frame_num'], sort=False)
    predictions = [df for _, df in predictions]
    
    # mapping annotations from prediction list to video/time
    frame_data = {}
    for i, group in enumerate(predictions):
        frame_num = group.iloc[0]['frame_num']
        video = group.iloc[0]['videoid']
        frame_data[(video,frame_num)] = i
    
    # group validation annotations by video frames
    validation = validation.groupby(['videoid','frame_num'], sort=False)
    validation = [df for _, df in validation]
    
    # initialize counters for each concept
    true_positives = dict(zip(concepts,[0] * len(concepts)))
    false_positives = dict(zip(concepts,[0] * len(concepts)))
    false_negatives = dict(zip(concepts,[0] * len(concepts)))
    
    # get true and false positives for each frame of validation data
    for group in validation:
        #get frame and video for this set
        frame_num = group.iloc[0]['frame_num']
        video = group.iloc[0]['videoid']
        
        #get corresponding predictions for this set
        try:
            predicted = predictions[frame_data[(video,frame_num)]]
        except:
            print("prediction not found")
            continue
            
        #go through predictions and identify false positives/false negatives/ true_positives
        detected_predictions = []
        detected_truths = dict(zip(concepts,[0] * len(concepts)))
        for index, truth in group.iterrows():
            for index, prediction in predicted.iterrows():
                if prediction.id in detected_predictions or compute_overlap(truth,prediction) < iou_thresh or prediction.conceptid != truth.conceptid:
                    continue
                detected_predictions.append(prediction.id)
                if truth.conceptid == prediction.conceptid:
                    true_positives[prediction.conceptid] += 1
                    detected_truths[prediction.conceptid] += 1
                else:
                    false_positives[prediction.conceptid] += 1

        for id in concepts:
            false_negatives[id] += len(group.loc[group['conceptid'] == id]) - detected_truths[id]
        #count number of annotations that were correct, wrong, and missed
        
    f1 = dict(zip(concepts,[0] * len(concepts)))
    for id in concepts:
        try:
            recall = true_positives[id] / (true_positives[id] + false_negatives[id])
            precision = true_positives[id] / (true_positives[id] + false_positives[id])
            f1[id] = 2*recall*precision / (precision+recall)
        except:
            f1[id] = 0
    
    return f1

In [37]:
results['videoid'] = 8
score_predictions(annotations,results,.5, concepts, fps)

KeyError: 'frame_num'

In [74]:
results.tail(30)

,id,x1,y1,x2,y2,label,confidence,objectid,frame_num,conceptid,videoid
270,63,20.0,268.000000,93.000000,337.000000,236,NaN,59e980c2-ae5d-4f79-b07a-7a7c4373f484,5063,236,8
271,64,18.0,273.000000,91.000000,342.000000,236,NaN,59e980c2-ae5d-4f79-b07a-7a7c4373f484,5064,236,8
272,65,17.0,280.000000,90.000000,349.000000,236,NaN,59e980c2-ae5d-4f79-b07a-7a7c4373f484,5065,236,8
273,66,16.0,282.000000,89.000000,351.000000,236,NaN,59e980c2-ae5d-4f79-b07a-7a7c4373f484,5066,236,8
274,67,13.0,288.000000,86.000000,357.000000,236,NaN,59e980c2-ae5d-4f79-b07a-7a7c4373f484,5067,236,8
275,68,13.0,290.000000,86.000000,359.000000,236,NaN,59e980c2-ae5d-4f79-b07a-7a7c4373f484,5068,236,8
276,69,11.0,297.000000,84.000000,366.000000,236,NaN,59e980c2-ae5d-4f79-b07a-7a7c4373f484,5069,236,8
277,70,9.0,301.000000,82.000000,370.000000,236,NaN,59e980c2-ae5d-4f79-b07a-7a7c4373f484,5070,236,8
278,71,7.0,309.000000,80.000000,378.000000,236,NaN,59e980c2-ae5d-4f79-b07a-7a7c4373f484,5071,236,8
279,72,5.0,313.000000,78.000000,382.000000,236,NaN,59e980c2-ae5d-4f79-b07a-7a7c4373f484,5072,236,8


In [72]:
annotations.head()

,id,videoid,userid,conceptid,timeinvideo,x1,y1,x2,y2,videowidth,videoheight,dateannotated,image,imagewithbox,comment,unsure,originalid,framenum,frame_num
0,411,8,6,236,169.627744,1198.006251,272.006241,1279.000000,377.999985,1280.0,720.0,2018-09-05,411.png,411_box.png,None,False,411,None,5084.0
1,414,8,6,236,169.647763,215.006236,96.006256,330.993731,199.993752,1280.0,720.0,2018-09-05,414.png,414_box.png,None,False,414,None,5084.0
2,413,8,6,236,169.647763,103.006251,157.006241,244.006236,281.993752,1280.0,720.0,2018-09-05,413.png,413_box.png,None,False,413,None,5084.0
3,410,8,6,237,169.627744,1057.006251,210.006241,1279.000000,358.993729,1280.0,720.0,2018-09-05,410.png,410_box.png,None,False,410,None,5084.0
4,404,8,6,383,160.694075,890.006251,100.006256,1166.006221,305.000015,1280.0,720.0,2018-09-05,404.png,404_box.png,None,False,404,None,4816.0
5,406,8,6,237,165.390121,411.006251,51.006256,607.993770,241.993744,1280.0,720.0,2018-09-05,406.png,406_box.png,None,False,406,None,4957.0
6,409,8,6,236,166.554130,93.006251,95.006241,225.006251,209.999985,1280.0,720.0,2018-09-05,409.png,409_box.png,None,False,409,None,4992.0
7,407,8,6,236,166.554130,163.006251,195.006241,266.993747,294.006233,1280.0,720.0,2018-09-05,407.png,407_box.png,None,False,407,None,4992.0
8,415,8,6,236,169.647763,961.006190,47.006256,1081.006183,150.993752,1280.0,720.0,2018-09-05,415.png,415_box.png,None,False,415,None,5084.0
9,412,8,6,236,169.647763,63.006251,246.006241,204.006251,370.993736,1280.0,720.0,2018-09-05,412.png,412_box.png,None,False,412,None,5084.0


In [ ]:
a = 32/29.97
a

In [ ]:
round(a,0)

In [6]:
queryDB("select * from concepts limit 10")

,id,name,rank,parent,picture
0,1,Bacteria,phylum,0,none.png
1,2,bacterial mat,species,1,bacterial_mat_02.png
2,3,Schizomycetes,class,1,none.png
3,4,Beggiatoales,order,3,none.png
4,5,Beggiatoaceae,family,4,none.png
5,6,Thioplaca,genus,5,none.png
6,7,Beggiatoa,genus,5,none.png
7,8,Cyanobacteria,phylum,0,none.png
8,9,Oscillatoriales,order,8,none.png
9,10,Oscillatoriacea,family,9,none.png


In [4]:
queryDB("select * from annotations limit 10")

,id,videoid,userid,conceptid,timeinvideo,x1,y1,x2,y2,videowidth,videoheight,dateannotated,image,imagewithbox,comment,unsure,originalid,framenum
0,2178619,19,17,2136,773.44,614.0,474.0,690.0,512.0,1280.0,720.0,2019-01-27,19_773.44_ai.png,6945_773.44_box_ai.png,None,False,6945,23180
1,2178626,19,17,2136,773.48,614.0,478.0,690.0,516.0,1280.0,720.0,2019-01-27,19_773.48_ai.png,6945_773.48_box_ai.png,None,False,6945,23181
2,2178641,19,17,2136,773.51,615.0,481.0,691.0,519.0,1280.0,720.0,2019-01-27,19_773.51_ai.png,6945_773.51_box_ai.png,None,False,6945,23182
3,2178664,19,17,2136,773.54,615.0,484.0,691.0,522.0,1280.0,720.0,2019-01-27,19_773.54_ai.png,6945_773.54_box_ai.png,None,False,6945,23183
4,2178694,19,17,2136,773.58,616.0,488.0,692.0,526.0,1280.0,720.0,2019-01-27,19_773.58_ai.png,6945_773.58_box_ai.png,None,False,6945,23184
5,2178724,19,17,2136,773.61,616.0,491.0,692.0,529.0,1280.0,720.0,2019-01-27,19_773.61_ai.png,6945_773.61_box_ai.png,None,False,6945,23185
6,2178746,19,17,2136,773.64,616.0,495.0,692.0,533.0,1280.0,720.0,2019-01-27,19_773.64_ai.png,6945_773.64_box_ai.png,None,False,6945,23186
7,2178765,19,17,2136,773.68,616.0,498.0,692.0,536.0,1280.0,720.0,2019-01-27,19_773.68_ai.png,6945_773.68_box_ai.png,None,False,6945,23187
8,2178784,19,17,2136,773.71,616.0,502.0,692.0,540.0,1280.0,720.0,2019-01-27,19_773.71_ai.png,6945_773.71_box_ai.png,None,False,6945,23188
9,2178804,19,17,2136,773.74,617.0,505.0,693.0,543.0,1280.0,720.0,2019-01-27,19_773.74_ai.png,6945_773.74_box_ai.png,None,False,6945,23189


In [5]:
queryDB("select * from users limit 10")

,id,username,password,admin
0,7,lubo,$2b$10$0aVOsMCvPElbyNbXBtBiQeImcuzKzztzSf/HDq1...,True
1,8,lubo93,$2b$10$R2baqvH5KEcTuJSO5ZR0JeHHkMNZweZJX11rC00...,False
2,9,sam,$2b$10$9YAnR5bZVnk.wapmDbaFDePTZbmp2KJVrVyzcNs...,True
3,10,sam2,$2b$10$cTBlJX8aQC8joysM78ZTtuAj5vW55Trwuy6kYe....,False
4,11,kellen,$2b$10$1N1SZuM.JxRCvaMUIWLw8ONXjNmWCL4af4A1ah9...,False
5,12,emma,$2b$10$u9qJDJwwaq0ctlH6AgY0i.HFlk7lSBNcrizC6zj...,False
6,6,ali,$2b$10$FY1plJa/tTnL3l6JbZpfh.bt9f2mLdLxsUX/0NU...,False
7,14,hanson,$2b$10$siRvJ9RowqSb/2ap.sTonO/Fn/Pe/HRj0ihBnMs...,True
8,13,adminAli,$2b$10$MEj63uH7AtaENBJCWBkiH.e8.X9NCxNc01DzC1i...,True
9,15,sierra_s,$2b$10$Y9VJS8E1Fi2tfF65du6IC.MZ2MhZ76F6xn5sDc3...,False
